In [3]:
import cv2
import os
import numpy as np
import tensorflow as tf

In [6]:
traindir = 'C:/Users/karte/OneDrive/Desktop/VIT/MLProjects/MyMLProjects/Emotion Detector/train'
testdir = 'C:/Users/karte/OneDrive/Desktop/VIT/MLProjects/MyMLProjects/Emotion Detector/test'
train_ds = tf.keras.utils.image_dataset_from_directory( #Using keras library to import testing and training dataset
  traindir,                                              #The images are already preprocessed, so im not doing any preprocessing
  image_size=(48,48)
)
test_ds = tf.keras.utils.image_dataset_from_directory(
  testdir,
  image_size=(48,48)
)
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255.0)

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [4]:
model = tf.keras.Sequential([
    normalization_layer,
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(48,48,3)), 
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7)
]
)
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
mymodel = model.fit(train_ds,validation_data=test_ds,epochs=20)

Epoch 1/20
898/898 [==============================] - 28s 30ms/step - loss: 1.6313 - accuracy: 0.3556 - val_loss: 1.5156 - val_accuracy: 0.4166
Epoch 2/20
898/898 [==============================] - 28s 31ms/step - loss: 1.4104 - accuracy: 0.4590 - val_loss: 1.3567 - val_accuracy: 0.4809
Epoch 3/20
898/898 [==============================] - 30s 33ms/step - loss: 1.3153 - accuracy: 0.4996 - val_loss: 1.3101 - val_accuracy: 0.4987
Epoch 4/20
898/898 [==============================] - 29s 33ms/step - loss: 1.2469 - accuracy: 0.5255 - val_loss: 1.3041 - val_accuracy: 0.4965
Epoch 5/20
898/898 [==============================] - 29s 32ms/step - loss: 1.1838 - accuracy: 0.5493 - val_loss: 1.2686 - val_accuracy: 0.5071
Epoch 6/20
898/898 [==============================] - 27s 30ms/step - loss: 1.1332 - accuracy: 0.5720 - val_loss: 1.2648 - val_accuracy: 0.5103
Epoch 7/20
898/898 [==============================] - 27s 30ms/step - loss: 1.0842 - accuracy: 0.5920 - val_loss: 1.2573 - val_accuracy:

In [2]:
model.save('./Saved Models/mymodel')

NameError: name 'model' is not defined

In [4]:
model = tf.keras.models.load_model('./Saved Models/mymodel')

In [9]:
import cv2 #the library used for image processing
import numpy as np
image = cv2.imread('im84.png')
resized_image = cv2.resize(image, (48, 48))  # Resize to match the input shape of your model
#normalized_image = resized_image / 255.0  # Normalize pixel values to the range of [0, 1]
preprocessed_image = np.expand_dims(resized_image, axis=0)  # Add an extra dimension for batch size (assuming single image) - idk
# Make a prediction
predictions = model.predict(preprocessed_image)

# Interpret the results
predicted_class = np.argmax(predictions)  # Get the index of the class with the highest probability
class_probabilities = predictions[0]  # Probability distribution across all classes
print(test_ds.class_names[predicted_class])
print(f"Class Probabilities: {class_probabilities}")

1/1 [==============================] - 0s 25ms/step
sad
Class Probabilities: [ 1.5189804 -2.681486   1.9173869 -3.9475846 -1.8220094  2.388909
 -7.4990273]
